# This script is DEPRECIATED! Keeping it around for code fragments only.

In [ ]:
from matplotlib import pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2


In [ ]:
import numpy as np
import pandas as pd

import sys
sys.path.append('..')

from ehd_dataset.utils import cell_to_array

DB = "C:/Dropbox/SPEED/Self Driving EHD/Datasets/10-Mar-2022 large nozzle mosaic/logs/measurements.xlsx"
DB = "C:/Dropbox/SPEED/Self Driving EHD/Datasets/29-Mar-2022 lg 1cm 300 points/logs/measurements.xlsx"
df = pd.read_excel(DB, index_col=0)

df.head()

features = ['absintegral', 'absmax', 'vmag', 'bias', 'maxcumsum']
F = np.array(df[features])

X = np.array([cell_to_array(x) for x in df.vector])
Y = np.array(df.area)

# for i in df.index:
#     df.wave.loc[i] = cell_to_array(df.wave.loc[i])

In [ ]:
# Initialize some ML kit
from scipy.stats import linregress

from sklearn import metrics
from sklearn.model_selection import KFold

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor


def calc_auc(Y, Y_true):
    thresh = 0
    auc = metrics.roc_auc_score(Y_true > thresh,
                                Y > thresh)
    return auc


def calc_mae(Y, Y_true):
    mae = np.abs(Y - Y_true).mean()
    return mae


def calc_mse(Y, Y_true):
    mse = ((Y - Y_true)**2).mean()
    return mse


def calc_r_val(Y, Y_true):
    # slope, intercept, r_value, p_value, std_err
    _, _, r_value, _, _ = linregress(Y, Y_true)
    return r_value


def norm(x):
    mean = x.mean(0)
    std = x.std(0)
    return (x - mean)/std, mean, std


class MLE_Regressor():
    def __init__(self):
        self.answers = None
    def fit(self, X, Y):
        self.answers = Y.mean(0)
    def predict(self, X):
        '''Unpredictable for multiple X!'''
        return self.answers


def LOO(X, Y, model, plot=False):
    N = Y.shape[0]
    guesses = np.zeros(Y.shape)
    for i in range(N):
        trainset = np.arange(N) != i
        X_train = X[trainset, :]
        Y_train = Y[trainset]
        # TODO stop overfitting!:
        # X_train = X
        # Y_train = Y
        model.fit(X_train, Y_train)
        guesses[i] = model.predict(X[i][None, :])
    if plot:
        plt.plot(Y, guesses, '*'); plt.xlabel('truth'); plt.ylabel('prediction')
#         plt.savefig('sin coef best model.png')
#         plt.savefig('custom feat best model.png')
        plt.show()
    return calc_mae(Y, guesses), calc_mse(Y, guesses), calc_r_val(Y, guesses), calc_auc(Y, guesses)


def eval_N_folds(X, Y, model, plot=False, N=10):
    kf = KFold(n_splits=N, shuffle=True, random_state=42)
    for train_index, test_index in kf.split(X):
        # TODO

Y, mean, std = norm(Y)

In [ ]:
results = pd.DataFrame(columns=('model', 'mae', 'mse', 'r', 'auc'))
models = {'MLE': MLE_Regressor(),
         'KNN': KNeighborsRegressor(),
         'Ridge': Ridge(),
         'Kernel': KernelRidge(),
         'Random Forest': RandomForestRegressor(),
         'MLP': MLPRegressor(max_iter=2000)}
best = 'Random Forest'

for label, model in models.items():
    print(f'Training {label}...')
    if label == best:
        print(f'{label} LOO results:')
        mae, mse, r_val, auc = LOO(X, Y, model, plot=True)
    else:
        mae, mse, r_val, auc = LOO(X, Y, model)
    print(f'{label} mae: {mae:.4f}   mse: {mse:.4f}   r: {r_val:.4f}   auc: {auc:.4f}', end='\n\n')
    results = results.append({'model': label, 'mae': mae, 'mse': mse, 'r': r_val, 'auc': auc},
                             ignore_index=True)
results.to_excel('training_summary.xlsx')

In [ ]:
results = pd.DataFrame(columns=('model', 'mae', 'mse', 'r', 'auc'))
models = {'MLE': MLE_Regressor(),
         'KNN': KNeighborsRegressor(),
         'Ridge': Ridge(),
         'Kernel': KernelRidge(),
         'Random Forest': RandomForestRegressor(),
         'MLP': MLPRegressor(max_iter=2000)}
best = 'Ridge'

for label, model in models.items():
    print(f'Training {label}...')
    if label == best:
        print(f'{label} LOO results:')
        mae, mse, r_val, auc = LOO(F, Y, model, plot=True)
    else:
        mae, mse, r_val, auc = LOO(F, Y, model)
    print(f'{label} mae: {mae:.4f}   mse: {mse:.4f}   r: {r_val:.4f}   auc: {auc:.4f}', end='\n\n')
    results = results.append({'model': label, 'mae': mae, 'mse': mse, 'r': r_val, 'auc': auc},
                             ignore_index=True)
results.to_excel('training_summary_customfeat.xlsx')

In [ ]:
results = pd.DataFrame(columns=('model', 'mae', 'mse', 'r', 'auc'))
models = {'MLE': MLE_Regressor(),
         'KNN': KNeighborsRegressor(),
         'Ridge': Ridge(),
         'Kernel': KernelRidge(),
         'Random Forest': RandomForestRegressor(),
         'MLP': MLPRegressor(max_iter=2000)}
best = 'Random Forest'

G = np.concatenate((X, F), 1)

for label, model in models.items():
    print(f'Training {label}...')
    if label == best:
        print(f'{label} LOO results:')
        mae, mse, r_val, auc = LOO(G, Y, model, plot=True)
    else:
        mae, mse, r_val, auc = LOO(G, Y, model)
    print(f'{label} mae: {mae:.4f}   mse: {mse:.4f}   r: {r_val:.4f}   auc: {auc:.4f}', end='\n\n')
    results = results.append({'model': label, 'mae': mae, 'mse': mse, 'r': r_val, 'auc': auc},
                             ignore_index=True)
results.to_excel('training_summary_combinedfeat.xlsx')